# Combine GEFS and Mesonet Data

Notebook to preprocess the GEFS data and combine it with the Mesonet solar station data, into a single data file for use in the modelling process

In [2]:
import xarray as xr
import pandas as pd

from collections import OrderedDict
from math import radians, cos, sin, asin, sqrt

In [3]:
!ls

Preprocess Data.ipynb Untitled.ipynb


### GEFS Prediction Data

Load the consolidated GEFS training data and create the new forecast_datetime feature (actual end date datetime corresponding to the numberical prediction for the prior 3 hour period back from the end date value)

In [2]:
gefs_train_df = pd.read_csv("../data/gefs/", index_col=0)
gefs_train_df.drop(["intTime", "intValidTime"], axis=1, inplace=True)
gefs_train_df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,ens,fhour,lat,lon,time,Total_precipitation,Precipitable_water,Pressure,Temperature_surface,Specific_humidity_height_above_ground,Downward_Short-Wave_Rad_Flux,Total_Column-Integrated_Condensate,Minimum_temperature,Upward_Long-Wave_Rad_Flux_surface,Maximum_temperature,Upward_Short-Wave_Rad_Flux,Total_cloud_cover,Downward_Long-Wave_Rad_Flux,Temperature_height_above_ground,Upward_Long-Wave_Rad_Flux
0,0,0 days 12:00:00.000000000,31.0,254.0,1994-01-01,0.0,5.700000,102244.94,275.60760,0.004020,0.0,0.0002,278.54382,333.76056,280.90344,0.0,0.0,247.01898,278.52792,255.59988
1,0,0 days 12:00:00.000000000,31.0,254.0,1994-01-02,0.0,6.798102,101962.01,276.50568,0.003630,0.0,0.0000,279.85657,336.21130,281.62622,0.0,0.0,257.01898,279.87485,252.67752
2,0,0 days 12:00:00.000000000,31.0,254.0,1994-01-03,0.0,4.891023,102341.00,272.48004,0.001580,0.0,0.0007,275.89343,318.73020,278.13170,0.0,0.0,231.84859,275.89554,242.08640
3,0,0 days 12:00:00.000000000,31.0,254.0,1994-01-04,0.0,4.944364,102438.59,274.79395,0.001930,0.0,0.0004,277.90198,332.61664,282.01617,0.0,0.0,241.16321,278.04550,254.67052
4,0,0 days 12:00:00.000000000,31.0,254.0,1994-01-05,0.0,4.567267,101882.70,274.73816,0.002092,0.0,0.0002,278.60968,329.19788,280.72284,0.0,0.0,242.19736,278.77753,254.01093


### GEFS Grid Elevation Data

Load the elevation data on each of the GEFS grid simulation points

In [3]:
gefs_elevation_df = xr.open_dataset("../data/gefs_data/gefs_elevations.nc").to_dataframe().reset_index()
gefs_elevation_df.head()

,lat,lon,elevation_control,elevation_perturbation,latitude,longitude
0,0,0,1420.699219,1342.618286,31.0,254.0
1,0,1,1346.989990,1328.494019,31.0,255.0
2,0,2,1323.471436,1244.265381,31.0,256.0
3,0,3,878.612122,919.849365,31.0,257.0
4,0,4,801.242798,821.093567,31.0,258.0


### Mesonet Solar Energy Readings

Load the historical daily Mesonet solar station energy reading

In [4]:
mesonet_readings_df = pd.read_csv("../data/train.csv")
mesonet_readings_df.head()

,Date,ACME,ADAX,ALTU,APAC,ARNE,BEAV,BESS,BIXB,BLAC,...,VINI,WASH,WATO,WAUR,WEAT,WEST,WILB,WIST,WOOD,WYNO
0,19940101,12384900,11930700,12116700,12301200,10706100,10116900,11487900,11182800,10848300,...,10771800,12116400,11308800,12361800,11331600,10644300,11715600,11241000,10490100,10545300
1,19940102,11908500,9778500,10862700,11666400,8062500,9262800,9235200,3963300,3318300,...,4314300,10733400,9154800,12041400,9168300,4082700,9228000,5829900,7412100,3345300
2,19940103,12470700,9771900,12627300,12782700,11618400,10789800,11895900,4512600,5266500,...,2976900,11775000,10700400,12687300,11324400,2746500,3686700,4488900,9712200,4442100
3,19940104,12725400,6466800,13065300,12817500,12134400,11816700,12186600,3212700,8270100,...,3476400,12159600,11907000,12953100,11903700,2741400,4905000,4089300,11401500,4365000
4,19940105,10894800,11545200,8060400,10379400,6918600,9936300,6411300,9566100,8009400,...,6393300,11419500,7334400,10178700,7471500,8235300,11159100,10651500,10006200,8568300


### Mesonet Station Coordinates

Load the information on each Mesonet solar station. As noted in the Kaggle challenge data description, we need to transform the longitude of the Mesonet station data to coincide with the same origin or meridian reference by adding 360 degrees to the existing longitude.

In [5]:
mesonet_stations_df = pd.read_csv("../data/station_info.csv")
mesonet_stations_df["elon_corrected"] = mesonet_stations_df["elon"] + 360
mesonet_stations_df.head()

,stid,nlat,elon,elev,elon_corrected
0,ACME,34.80833,-98.02325,397,261.97675
1,ADAX,34.79851,-96.66909,295,263.33091
2,ALTU,34.58722,-99.33808,416,260.66192
3,APAC,34.91418,-98.29216,440,261.70784
4,ARNE,36.07204,-99.90308,719,260.09692


### Combining Mesonet and GEFS Data

For simiplicity, we'll link together the Mesonet and GEFS data by matching each Mesonet station together with the closest GEFS grid point. The closest GEFS grid point will be determined by using the haversine formula to compute the great circle distance between each Mesonet station and each GEFS grid point, and choosing the closest grid point with the minimal distance. 

In [6]:
def df_crossjoin(df1_, df2_, **kwargs):
    """
    Author: Markus Konrad <post@mkonrad.net>
    April 2016
    Make a cross join (cartesian product) between two dataframes by using a constant temporary key.
    Also sets a MultiIndex which is the cartesian product of the indices of the input dataframes.
    See: https://github.com/pydata/pandas/issues/5401
    :param df1 dataframe 1
    :param df1 dataframe 2
    :param kwargs keyword arguments that will be passed to pd.merge()
    :return cross join of df1 and df2
    """
    df1 = df1_.copy(deep=True)
    df2 = df2_.copy(deep=True)
    
    df1['_tmpkey'] = 1
    df2['_tmpkey'] = 1

    res = pd.merge(df1, df2, on='_tmpkey', **kwargs).drop('_tmpkey', axis=1)
    res.index = pd.MultiIndex.from_product((df1.index, df2.index))

    df1.drop('_tmpkey', axis=1, inplace=True)
    df2.drop('_tmpkey', axis=1, inplace=True)

    return res


def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [7]:
gefs_grid_coords_df = gefs_elevation_df[["latitude", "longitude", "elevation_control", "elevation_perturbation"]]
gefs_grid_coords_df.columns = ["gefs_lat", "gefs_lon", "elevation_control", "elevation_perturbation"]

Form the cartesian production between the coordinates of all the Mesonet stations and the GEFS grid points

In [8]:
distances_df = df_crossjoin(
    mesonet_stations_df, 
    gefs_grid_coords_df, 
    suffixes=('_orig', '_dest')
).reset_index(drop=True)
distances_df.head()

,stid,nlat,elon,elev,elon_corrected,gefs_lat,gefs_lon,elevation_control,elevation_perturbation
0,ACME,34.80833,-98.02325,397,261.97675,31.0,254.0,1420.699219,1342.618286
1,ACME,34.80833,-98.02325,397,261.97675,31.0,255.0,1346.989990,1328.494019
2,ACME,34.80833,-98.02325,397,261.97675,31.0,256.0,1323.471436,1244.265381
3,ACME,34.80833,-98.02325,397,261.97675,31.0,257.0,878.612122,919.849365
4,ACME,34.80833,-98.02325,397,261.97675,31.0,258.0,801.242798,821.093567


Compute all the great circle distances between the stations and grid points

In [9]:
distances_df["distance_km"] = distances_df.apply(
    lambda x: haversine(
        x["elon_corrected"],
        x["nlat"],
        x["gefs_lon"],
        x["gefs_lat"]
    ), 
    axis=1
)
distances_df.head()

,stid,nlat,elon,elev,elon_corrected,gefs_lat,gefs_lon,elevation_control,elevation_perturbation,distance_km
0,ACME,34.80833,-98.02325,397,261.97675,31.0,254.0,1420.699219,1342.618286,856.245389
1,ACME,34.80833,-98.02325,397,261.97675,31.0,255.0,1346.989990,1328.494019,776.560630
2,ACME,34.80833,-98.02325,397,261.97675,31.0,256.0,1323.471436,1244.265381,700.224888
3,ACME,34.80833,-98.02325,397,261.97675,31.0,257.0,878.612122,919.849365,628.467824
4,ACME,34.80833,-98.02325,397,261.97675,31.0,258.0,801.242798,821.093567,563.050404


For each Mesonet station, select the closest grid point

In [10]:
def get_closest_grid_point(group):
    "Find the closest grid point for a given station group"
    
    group_sorted = group.sort_values("distance_km", ascending=False)
    
    ret = pd.Series(OrderedDict([
        ("meso_lat", group_sorted.iloc[0]["nlat"]),
        ("meso_lon", group_sorted.iloc[0]["elon_corrected"]),
        ("meso_elev", group_sorted.iloc[0]["elev"]),
        ("gefs_lat", group_sorted.iloc[0]["gefs_lat"]),
        ("gefs_lon", group_sorted.iloc[0]["gefs_lon"]),
        ("gefs_elev", group_sorted.iloc[0]["elevation_control"]),
        ("distance_km", group_sorted.iloc[0]["distance_km"])
    ]))
    
    return ret
    

mesonet_gefs_matches_df = (
    distances_df
    .groupby("stid")
    .apply(lambda x: get_closest_grid_point(x))
).reset_index()
mesonet_gefs_matches_df.head()

,stid,meso_lat,meso_lon,meso_elev,gefs_lat,gefs_lon,gefs_elev,distance_km
0,ACME,34.80833,261.97675,397.0,31.0,254.0,1420.699219,856.245389
1,ADAX,34.79851,263.33091,295.0,31.0,254.0,1420.699219,967.568287
2,ALTU,34.58722,260.66192,416.0,39.0,269.0,195.383102,889.345616
3,APAC,34.91418,261.70784,440.0,31.0,254.0,1420.699219,840.192442
4,ARNE,36.07204,260.09692,719.0,31.0,269.0,84.810104,998.778385


With the mapping in hand, now join the Mesonet data back with the GEFS data. First, we need to reorient the Mesonet solar readings data.

In [11]:
mesonet_readings_melt_df = pd.melt(mesonet_readings_df, id_vars=['Date'], var_name='station', value_name='value')
mesonet_readings_melt_df.columns = ["datetime", "station", "measured_solar_output"]
mesonet_readings_melt_df.head()

,datetime,station,measured_solar_output
0,19940101,ACME,12384900
1,19940102,ACME,11908500
2,19940103,ACME,12470700
3,19940104,ACME,12725400
4,19940105,ACME,10894800


We'll want to join the Mesonet readings with the GEFS using the time variable. Let's reformat that in the GEFS data to be in the same format as the Mesonet data. This can be a little slow. 

In [12]:
gefs_train_df["datetime"] = pd.to_datetime(gefs_train_df["time"]).dt.strftime("%Y%m%d")
gefs_train_df.head()

,ens,fhour,lat,lon,time,Total_precipitation,Precipitable_water,Pressure,Temperature_surface,Specific_humidity_height_above_ground,...,Total_Column-Integrated_Condensate,Minimum_temperature,Upward_Long-Wave_Rad_Flux_surface,Maximum_temperature,Upward_Short-Wave_Rad_Flux,Total_cloud_cover,Downward_Long-Wave_Rad_Flux,Temperature_height_above_ground,Upward_Long-Wave_Rad_Flux,datetime
0,0,0 days 12:00:00.000000000,31.0,254.0,1994-01-01,0.0,5.700000,102244.94,275.60760,0.004020,...,0.0002,278.54382,333.76056,280.90344,0.0,0.0,247.01898,278.52792,255.59988,19940101
1,0,0 days 12:00:00.000000000,31.0,254.0,1994-01-02,0.0,6.798102,101962.01,276.50568,0.003630,...,0.0000,279.85657,336.21130,281.62622,0.0,0.0,257.01898,279.87485,252.67752,19940102
2,0,0 days 12:00:00.000000000,31.0,254.0,1994-01-03,0.0,4.891023,102341.00,272.48004,0.001580,...,0.0007,275.89343,318.73020,278.13170,0.0,0.0,231.84859,275.89554,242.08640,19940103
3,0,0 days 12:00:00.000000000,31.0,254.0,1994-01-04,0.0,4.944364,102438.59,274.79395,0.001930,...,0.0004,277.90198,332.61664,282.01617,0.0,0.0,241.16321,278.04550,254.67052,19940104
4,0,0 days 12:00:00.000000000,31.0,254.0,1994-01-05,0.0,4.567267,101882.70,274.73816,0.002092,...,0.0002,278.60968,329.19788,280.72284,0.0,0.0,242.19736,278.77753,254.01093,19940105


First combine the Mesonet solar readings and the matched GEFS grid points

In [13]:
tmp_1_df = pd.merge(
    mesonet_readings_melt_df,
    mesonet_gefs_matches_df,
    left_on="station",
    right_on="stid",
    how="left"
)
tmp_1_df.head()

,datetime,station,measured_solar_output,stid,meso_lat,meso_lon,meso_elev,gefs_lat,gefs_lon,gefs_elev,distance_km
0,19940101,ACME,12384900,ACME,34.80833,261.97675,397.0,31.0,254.0,1420.699219,856.245389
1,19940102,ACME,11908500,ACME,34.80833,261.97675,397.0,31.0,254.0,1420.699219,856.245389
2,19940103,ACME,12470700,ACME,34.80833,261.97675,397.0,31.0,254.0,1420.699219,856.245389
3,19940104,ACME,12725400,ACME,34.80833,261.97675,397.0,31.0,254.0,1420.699219,856.245389
4,19940105,ACME,10894800,ACME,34.80833,261.97675,397.0,31.0,254.0,1420.699219,856.245389


Before we combine the Mesonet and GEFS data, we need to make the GEFS data be on the daily level

In [14]:
gefs_daily_train_df = gefs_train_df.groupby(["ens","lat","lon","datetime"]).agg({"min","max","mean","sum","std"}).reset_index()
gefs_daily_train_df.columns = ['_'.join(col).strip().replace(" ","") for col in gefs_daily_train_df.columns.values]

In [15]:
gefs_daily_train_df.head()

,ens_,lat_,lon_,datetime_,Total_precipitation_std,Total_precipitation_mean,Total_precipitation_max,Total_precipitation_sum,Total_precipitation_min,Precipitable_water_std,...,Temperature_height_above_ground_std,Temperature_height_above_ground_mean,Temperature_height_above_ground_max,Temperature_height_above_ground_sum,Temperature_height_above_ground_min,Upward_Long-Wave_Rad_Flux_std,Upward_Long-Wave_Rad_Flux_mean,Upward_Long-Wave_Rad_Flux_max,Upward_Long-Wave_Rad_Flux_sum,Upward_Long-Wave_Rad_Flux_min
0,0,31.0,254.0,19940101,0.0,0.0,0.0,0.0,0.0,0.463681,...,3.630803,282.062242,286.58896,1410.31121,278.14370,12.204579,262.698988,277.00550,1313.49494,251.08438
1,0,31.0,254.0,19940102,0.0,0.0,0.0,0.0,0.0,1.875812,...,3.823121,283.273098,288.22590,1416.36549,279.00568,9.146205,262.872116,273.71368,1314.36058,252.67752
2,0,31.0,254.0,19940103,0.0,0.0,0.0,0.0,0.0,0.182287,...,4.994535,281.688362,287.18292,1408.44181,275.89554,19.363727,256.279414,277.83658,1281.39707,238.27545
3,0,31.0,254.0,19940104,0.0,0.0,0.0,0.0,0.0,0.415925,...,4.503648,282.617956,287.43414,1413.08978,277.57608,14.936433,262.229874,279.40970,1311.14937,246.53894
4,0,31.0,254.0,19940105,0.0,0.0,0.0,0.0,0.0,0.260593,...,6.719328,286.158764,293.53345,1430.79382,278.77753,15.572198,266.030710,283.45580,1330.15355,251.75024


Now combine the daily GEFS data with the daily Mesonet data, using the coorindates and timestamp from the matching

In [19]:
gefs_daily_train_df.to_csv("../data/gefs_data/gefs_train/gefs_daily.csv", index=False)
tmp_1_df.to_csv("../data/matched_mesonet_gefs.csv", index=False)

In [18]:
training_df = pd.merge(
    gefs_daily_train_df,
    tmp_1_df[["datetime","station","distance_km","gefs_lat","gefs_lon"]],
    left_on=["lat_", "lon_", "datetime_"],
    right_on=["gefs_lat", "gefs_lon", "datetime"],
    how="inner"
)

MemoryError: 